This script takes the quantity of items sold over a month, removes any items that haven't been seeling hardly at all (<=5 sales) and returns the bottom 3 remaining items. These ideally are items that people like, but maybe need some promoting.

In [1]:
#bring in relevant libraries
import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import httplib, urllib
import re
import requests
import datetime
import time
from datetime import datetime, timedelta
import httplib, urllib, json, locale
from urlparse import urlparse
from pymongo import MongoClient
from bson.objectid import ObjectId

merchant_ids = []
#connecting to our db
connection = MongoClient(host = '')
db=
merchant = db['Merchants']
for merchants in merchant.find():
     merchant_ids.append(str(merchants['_id']))

for MERCHANT_ID in merchant_ids:
    #eventually we'll want this to be brought in externally from mongo

    #object_id = 'ObjectId(\"' + MERCHANT_ID + '\")'
    doc1 = []

    for doc in merchant.find({'_id':ObjectId(MERCHANT_ID)}):
         doc1.append(doc)
    try:
        access_token = doc1[0]['pos_access_token']

        record=db['Reports']

        access_token = str(access_token)
    #except:
        #pass

        # Generating a payment report with the Square Connect API.


        # Get this from your application dashboard (https://connect.squareup.com/apps)
        # this is secret, don't share

        # Standard HTTP headers for every Connect API request
        request_headers = {'Authorization': 'Bearer ' + access_token,
                           'Accept': 'application/json',
                           'Content-Type': 'application/json'}

        # The base URL for every Connect API request
        connection = httplib.HTTPSConnection('connect.squareup.com')

        # Uses the locale to format currency amounts correctly
        locale.setlocale(locale.LC_ALL, 'en_US')

        # Helper function to convert cent-based money amounts to dollars and cents
        def format_money(amount):
          return locale.currency(amount / 100.)

        loc_ids = []
        # Obtains all of the business's location IDs. Each location has its own collection of payments.
        def get_location_ids():
          request_path = '/v1/me/locations'
          connection.request('GET', request_path, '', request_headers)
          response = connection.getresponse()

          # Transform the JSON array of locations into a Python list
          locations = json.loads(response.read())

          location_ids = []
          for location in locations:
            location_ids.append(location['id'])
            loc_ids.append(location['id'])
          return location_ids


        # Downloads all of a business's payments
        def get_payments(location_ids):

        #get time right now, should be correct if computer is synced properly
          d = datetime.utcnow()
        #we want data for the last 12 hours (this is a safe bet, because it'll only capture the business hour's for one day inherently)
          beg_time = datetime.utcnow() - timedelta(hours = 12)


        #turn current time into a bunch of strings that we can add together to form link parameters below
          year,month,day,hour,minute,second = str(d.year),str(d.month),str(d.day),str(d.hour),str(d.minute),str(d.second)
          year_beg,month_beg,day_beg,hour_beg,minute_beg,second_beg = str(beg_time.year),str(beg_time.month),str(beg_time.day),str(beg_time.hour),str(beg_time.minute),str(beg_time.second)

        #if any of our dates/times are only one digit, we add a '0' to the front to make them suitable for URL 
          if len(day) == 1:
                day = '0'+ day
          if len(month) == 1:
                month = '0'+ month
          if len(hour) == 1:
                hour = '0' + hour
          if len(minute) == 1:
                minute = '0' + minute
          if len(second) == 1:
                second = '0' + second

          if len(day_beg) == 1:
                    day_beg = '0'+ day_beg
          if len(month_beg) == 1:
                    month_beg = '0'+ month_beg
          if len(hour_beg) == 1:
                hour_beg = '0' + hour_beg
          if len(minute_beg) == 1:
                minute_beg = '0' + minute_beg
          if len(second_beg) == 1:
                second_beg = '0' + second_beg  

          parameters = urllib.urlencode({'begin_time': year_beg + '-' + month_beg + '-' + day_beg + 'T' + hour_beg + ':' + minute_beg + ':' + '00',
                                         'end_time'  : year + '-' + month + '-' + day + 'T' + hour + ':' + minute + ':' + '00'})
          payments = []

          # For each location...
          for location_id in location_ids:

            print 'Downloading payments for location with ID ' + location_id + '...'

            request_path = '/v1/' + location_id + '/payments?' + parameters
            more_results = True

            # ...as long as there are more payments to download from the location...
            while more_results:

              # ...send a GET request to /v1/LOCATION_ID/payments
              connection.request('GET', request_path, '', request_headers)
              response = connection.getresponse()

              # Read the response body JSON into the cumulative list of results
              payments = payments + json.loads(response.read())

              # Check whether pagination information is included in a response header, indicating more results
              pagination_header = response.getheader('link', '')
              if "rel='next'" not in pagination_header:
                more_results = False
              else:

                # Extract the next batch URL from the header.
                #
                # Pagination headers have the following format:
                # <https://connect.squareup.com/v1/LOCATION_ID/payments?batch_token=BATCH_TOKEN>;rel='next'
                # This line extracts the URL from the angle brackets surrounding it.
                next_batch_url = urlparse(pagination_header.split('<')[1].split('>')[0])

                request_path = next_batch_url.path + '?' + next_batch_url.query

          # Remove potential duplicate values from the list of payments
          seen_payment_ids = set()
          unique_payments = []

          for payment in payments:
            if payment['id'] in seen_payment_ids:
              continue
            seen_payment_ids.add(payment['id'])
            unique_payments.append(payment)

          return unique_payments


        if __name__ == '__main__':

          # Get all payments from all of the business's locations
          payments = get_payments(get_location_ids())
          pay_data = payments
          # Print a sales summary report of the payments
          #print_sales_report(payments)
          connection.close()










        # The base URL for every Connect API request
        connection = httplib.HTTPSConnection('connect.squareup.com')

        # Uses the locale to format currency amounts correctly
        locale.setlocale(locale.LC_ALL, 'en_US')

        # Helper function to convert cent-based money amounts to dollars and cents
        def format_money(amount):
          return locale.currency(amount / 100.)


        # Obtains all of the business's location IDs. Each location has its own collection of payments.
        def get_location_ids():
          request_path = '/v1/me/locations'
          connection.request('GET', request_path, '', request_headers)
          response = connection.getresponse()

          # Transform the JSON array of locations into a Python list
          locations = json.loads(response.read())

          location_ids = []
          for location in locations:
            location_ids.append(location['id'])

          return location_ids


        # Downloads all of a business's payments
        def get_payments(location_ids):

        #get time right now, timezone should be correct if computer is synced properly
          d = datetime.utcnow()
        #we want data from the last hour of today - 1 week
          diff_time = datetime.utcnow() - timedelta(weeks = 1)
          beg_time = diff_time - timedelta(hours = 12)


        #turn current time into a bunch of strings that we can add together to form link parameters below
          year_diff,month_diff,day_diff,hour_diff,minute_diff,second_diff = str(diff_time.year),str(diff_time.month),str(diff_time.day),str(diff_time.hour),str(diff_time.minute),str(diff_time.second)
          year_beg,month_beg,day_beg,hour_beg,minute_beg,second_beg = str(beg_time.year),str(beg_time.month),str(beg_time.day),str(beg_time.hour),str(beg_time.minute),str(beg_time.second)

        #if any of our dates/times are only one digit, we add a '0' to the front to make them suitable for URL 
          if len(day_diff) == 1:
                day_diff = '0'+ day_diff
          if len(month_diff) == 1:
                month_diff = '0'+ month_diff
          if len(hour_diff) == 1:
                hour_diff = '0' + hour_diff
          if len(minute_diff) == 1:
                minute_diff = '0' + minute_diff
          if len(second_diff) == 1:
                second_diff = '0' + second_diff

          if len(day_beg) == 1:
                day_beg = '0'+ day_beg
          if len(month_beg) == 1:
                month_beg = '0'+ month_beg
          if len(hour_beg) == 1:
                hour_beg = '0' + hour_beg
          if len(minute_beg) == 1:
                minute_beg = '0' + minute_beg
          if len(second_beg) == 1:
                second_beg = '0' + second_beg

          parameters = urllib.urlencode({'begin_time': year_beg + '-' + month_beg + '-' + day_beg + 'T' + hour_beg +':'+ minute_beg + ':' + '00',
                                         'end_time'  : year_diff + '-' + month_diff + '-' + day_diff + 'T' + hour_diff + ':' + minute_diff + ':' + '00'})
          payments = []

          # For each location...
          for location_id in location_ids:

            print 'Downloading payments for location with ID ' + location_id + '...'

            request_path = '/v1/' + location_id + '/payments?' + parameters
            more_results = True

            # ...as long as there are more payments to download from the location...
            while more_results:

              # ...send a GET request to /v1/LOCATION_ID/payments
              connection.request('GET', request_path, '', request_headers)
              response = connection.getresponse()

              # Read the response body JSON into the cumulative list of results
              payments = payments + json.loads(response.read())

              # Check whether pagination information is included in a response header, indicating more results
              pagination_header = response.getheader('link', '')
              if "rel='next'" not in pagination_header:
                more_results = False
              else:

                # Extract the next batch URL from the header.
                #
                # Pagination headers have the following format:
                # <https://connect.squareup.com/v1/LOCATION_ID/payments?batch_token=BATCH_TOKEN>;rel='next'
                # This line extracts the URL from the angle brackets surrounding it.
                next_batch_url = urlparse(pagination_header.split('<')[1].split('>')[0])

                request_path = next_batch_url.path + '?' + next_batch_url.query

          # Remove potential duplicate values from the list of payments
          seen_payment_ids = set()
          unique_payments = []

          for payment in payments:
            if payment['id'] in seen_payment_ids:
              continue
            seen_payment_ids.add(payment['id'])
            unique_payments.append(payment)

          return unique_payments


        if __name__ == '__main__':

          # Get all payments from all of the business's locations
          payments = get_payments(get_location_ids())
          pay_data_past = payments
          # Print a sales summary report of the payments
          #print_sales_report(payments)

          connection.close()





        # Generating a payment report with the Square Connect API.


        # Get this from your application dashboard (https://connect.squareup.com/apps)
        # this is secret, don't share

        # Standard HTTP headers for every Connect API request
        request_headers = {'Authorization': 'Bearer ' + access_token,
                           'Accept': 'application/json',
                           'Content-Type': 'application/json'}

        # The base URL for every Connect API request
        connection = httplib.HTTPSConnection('connect.squareup.com')

        # Uses the locale to format currency amounts correctly
        locale.setlocale(locale.LC_ALL, 'en_US')

        # Helper function to convert cent-based money amounts to dollars and cents
        def format_money(amount):
          return locale.currency(amount / 100.)


        # Obtains all of the business's location IDs. Each location has its own collection of payments.
        def get_location_ids():
          request_path = '/v1/me/locations'
          connection.request('GET', request_path, '', request_headers)
          response = connection.getresponse()

          # Transform the JSON array of locations into a Python list
          locations = json.loads(response.read())

          location_ids = []
          for location in locations:
            location_ids.append(location['id'])

          return location_ids


        # Downloads all of a business's payments for the last 7 days
        def get_payments(location_ids):
            for week_num in range(7):
            #get time right now, timezone should be correct if computer is synced properly
              d = datetime.utcnow()
            #we want all the data from the last week over the last hour
              diff_time = datetime.utcnow() - timedelta(days = int('{}'.format(week_num)))
              beg_time = diff_time - timedelta(hours = 12)


            #turn current time into a bunch of strings that we can add together to form link parameters below
              year_diff,month_diff,day_diff,hour_diff,minute_diff,second_diff = str(diff_time.year),str(diff_time.month),str(diff_time.day),str(diff_time.hour),str(diff_time.minute),str(diff_time.second)
              year_beg,month_beg,day_beg,hour_beg,minute_beg,second_beg = str(beg_time.year),str(beg_time.month),str(beg_time.day),str(beg_time.hour),str(beg_time.minute),str(beg_time.second)

            #if any of our dates/times are only one digit, we add a '0' to the front to make them suitable for URL 
              if len(day_diff) == 1:
                    day_diff = '0'+ day_diff
              if len(month_diff) == 1:
                    month_diff = '0'+ month_diff
              if len(hour_diff) == 1:
                    hour_diff = '0' + hour_diff
              if len(minute_diff) == 1:
                    minute_diff = '0' + minute_diff
              if len(second_diff) == 1:
                    second_diff = '0' + second_diff

              if len(day_beg) == 1:
                    day_beg = '0'+ day_beg
              if len(month_beg) == 1:
                    month_beg = '0'+ month_beg
              if len(hour_beg) == 1:
                    hour_beg = '0' + hour_beg
              if len(minute_beg) == 1:
                    minute_beg = '0' + minute_beg
              if len(second_beg) == 1:
                    second_beg = '0' + second_beg

              parameters = urllib.urlencode({'begin_time': year_beg + '-' + month_beg + '-' + day_beg + 'T' + hour_beg +':'+ minute_beg + ':' + '00',
                                             'end_time'  : year_diff + '-' + month_diff + '-' + day_diff + 'T' + hour_diff + ':' + minute_diff + ':' + '00'})
              payments = []

              # For each location...
              for location_id in location_ids:

                print 'Downloading payments for location with ID ' + location_id + '...'

                request_path = '/v1/' + location_id + '/payments?' + parameters
                more_results = True

                # ...as long as there are more payments to download from the location...
                while more_results:

                  # ...send a GET request to /v1/LOCATION_ID/payments
                  connection.request('GET', request_path, '', request_headers)
                  response = connection.getresponse()

                  # Read the response body JSON into the cumulative list of results
                  payments = payments + json.loads(response.read())

                  # Check whether pagination information is included in a response header, indicating more results
                  pagination_header = response.getheader('link', '')
                  if "rel='next'" not in pagination_header:
                    more_results = False
                  else:

                    # Extract the next batch URL from the header.
                    #
                    # Pagination headers have the following format:
                    # <https://connect.squareup.com/v1/LOCATION_ID/payments?batch_token=BATCH_TOKEN>;rel='next'
                    # This line extracts the URL from the angle brackets surrounding it.
                    next_batch_url = urlparse(pagination_header.split('<')[1].split('>')[0])

                    request_path = next_batch_url.path + '?' + next_batch_url.query

              # Remove potential duplicate values from the list of payments
              seen_payment_ids = set()
              unique_payments = []
              avg_list = []
              for payment in payments:
                if payment['id'] in seen_payment_ids:
                  continue
                seen_payment_ids.add(payment['id'])
                unique_payments.append(payment)
                avg_list.append(unique_payments)
              return avg_list


        if __name__ == '__main__':

          # Get all 2015 payments from all of the business's locations
          payments = get_payments(get_location_ids())
          #pay_data_avg is a list of jsons in this case
          pay_data_avg = payments
          # Print a sales summary report of the payments
          #print_sales_report(payments)

          connection.close()




        #list items, list payments

        # All requests to the Square Connect API require an access token in an
        # Authorization header. Specify your application's personal access token here
        # (available from https://connect.squareup.com/apps)

        # In addition to an Authorization header, requests to the Connect API should
        # include the indicated Accept and Content-Type headers.


        # Send a GET request to the ListLocations endpoint and obtain the response.
        connection = httplib.HTTPSConnection('connect.squareup.com')
        request_path = '/v1/'+loc_ids[0]+'/items'
        connection.request('GET', request_path, '', request_headers)
        response = connection.getresponse()



        # Convert the returned JSON body into an array of locations you can work with.
        items_data = json.loads(response.read())

        # Pretty-print the locations array.
        #locations = json.dumps(locations, indent=2, separators=(',',': '))


        #making empty lists to fill with JSON pulled data
        name_list = []
        quant_list = []
        quant_id_list = []
        time_list = []
        #pulling in item name,quantity purchased for the day, id of item, and time of purchase
        for i in pay_data:
            for n in range(10):
                #all of our entries should be complete every time, if not, we'll add a nan
                try:
                    name_list.append(i['itemizations'][n]['name'])
                except:
                    name_list.append(np.nan)
                try:
                    quant_list.append(str(i['itemizations'][n]['quantity']))
                except:
                    quant_list.append(0)
                try:
                    quant_id_list.append(i['itemizations'][n]['item_detail']['item_id'])
                except:
                    quant_id_list.append(np.nan)
                try:
                    time_list.append(i['created_at'])
                except:
                    time_list.append(np.nan)



        name_list_past = []
        quant_list_past = []
        quant_id_list_past = []
        time_list_past = []
        #pulling in item name,quantity purchased for the day, id of item, and time of purchase
        for i in pay_data_past:
            for n in range(10):
                #all of our entries should be complete every time, if not, we'll add a nan
                try:
                    name_list_past.append(i['itemizations'][n]['name'])
                except:
                    name_list_past.append(np.nan)
                try:
                    quant_list_past.append(str(i['itemizations'][n]['quantity']))
                except:
                    quant_list_past.append(0)
                try:
                    quant_id_list_past.append(i['itemizations'][n]['item_detail']['item_id'])
                except:
                    quant_id_list_past.append(np.nan)
                try:
                    time_list_past.append(i['created_at'])
                except:
                    time_list_past.append(np.nan)

        name_list_avg = []
        quant_list_avg = []
        quant_id_list_avg = []
        time_list_avg = []
        #pulling in item name,quantity purchased for the day, id of item, and time of purchase
        #since it's a list, we need to iterate over each element of the list as well as the actual json files
        for el in range(len(pay_data_avg)):  
            for i in pay_data_avg[el]:
                for n in range(10):
                    #all of our entries should be complete every time, if not, we'll add a nan
                    try:
                        name_list_avg.append(i['itemizations'][n]['name'])
                    except:
                        name_list_avg.append(np.nan)
                    try:
                        quant_list_avg.append(str(i['itemizations'][n]['quantity']))
                    except:
                        quant_list_avg.append(0)
                    try:
                        quant_id_list_avg.append(i['itemizations'][n]['item_detail']['item_id'])
                    except:
                        quant_id_list_avg.append(np.nan)
                    try:
                        time_list_avg.append(i['created_at'])
                    except:
                        time_list_avg.append(np.nan)
        
        
        #if our quantity for today and 7 days ago today is 0, the restaurant likely is not open, so we won't analyze
        #if np.sum(quant_list) == 0 and np.sum(quant_list_past) == 0:
         #   print (np.sum(quant_list), np.sum(quant_list_past))
          #  continue

        id_list = []
        url_list = []
        names_list = []
        price_list = []
        #pulling in item, id, price of item, and image url
        for m in items_data:
            #try and pull in data, if it doesn't exist for the instance, add a nan to be handled later
            try:      
                names_list.append(m['name'])
            except:
                names_list.append(np.nan)
            try:
                id_list.append(m['id'])
            except:
                id_list.append(np.nan)
            try:
                url_list.append(m['master_image']['url'])
            except:
                url_list.append(np.nan)
            try:
                price_list.append(m['variations'][0]['price_money']['amount'])
            except:
                price_list.append(np.nan)




        #create data frame for each item on the menu and all the relevant information about it
        df2 = pd.DataFrame({'id':id_list,'url':url_list, 'name':names_list, 'price':price_list})
        df2.price = df2.price/100
        df2_name_url= df2[['name','url']]
        df2_name_url.url[pd.isnull(df2_name_url.url)] = ''

        #the items without a url are not typical menu items, so we get rid of those entries.
        df2 = df2.dropna(subset = ['name']).reset_index()


        #Compile the payment data into a dataframe
        df_day = pd.DataFrame({'quantity':quant_list,'id':quant_id_list,'item_name':name_list,'time_of_sale':time_list})
        df_past = pd.DataFrame({'quantity':quant_list_past,'id':quant_id_list_past,'item_name':name_list_past,'time_of_sale':time_list_past})
        df_avg = pd.DataFrame({'quantity':quant_list_avg,'id':quant_id_list_avg,'item_name':name_list_avg,'time_of_sale':time_list_avg})


        #mergethe dataframes on ID, but maintain
        fin_df_day = df_day.merge(df2, on = 'id', how = 'outer')
        fin_df_past = df_past.merge(df2, on = 'id', how = 'outer')
        fin_df_avg = df_avg.merge(df2,on = 'id', how = 'outer')

        #we aren't interest in all of the columns from the original two dataframe, so we take the ones we want
        fin_df_day = fin_df_day[['name','quantity','id','price','url','time_of_sale']]
        fin_df_past = fin_df_past[['name','quantity','id','price','url','time_of_sale']]
        fin_df_avg = fin_df_avg[['name','quantity','id','price','url','time_of_sale']]

        #we are turning quantity into a numeric so we can aggregate it to get totala quantity sold in our group by below
        fin_df_day.quantity = pd.to_numeric(fin_df_day.loc[:,'quantity'])
        fin_df_past.quantity = pd.to_numeric(fin_df_past.loc[:,'quantity'])
        fin_df_avg.quantity = pd.to_numeric(fin_df_avg.loc[:,'quantity'])


        #groupby quantity to get total sold for each product
        fin_df_day = fin_df_day.groupby(['name','id']).sum().reset_index().sort_values('quantity', ascending = False)
        fin_df_past = fin_df_past.groupby(['name','id']).sum().reset_index().sort_values('quantity', ascending = False)
        fin_df_avg = fin_df_avg.groupby(['name','id']).sum().reset_index().sort_values('quantity', ascending = False)
        

        fin_df_day = fin_df_day.reset_index(drop=True)
        fin_df_past = fin_df_past.reset_index(drop=True)
        fin_df_avg = fin_df_avg.reset_index(drop=True)

        #bring together our today df and the last week one so we can compare
        fin_df = fin_df_day.merge(fin_df_past, on = ['name','id'], suffixes = ['_today','_week_ago'])

        #turn our nulls into 0's so we can use them in math operations
        fin_df.quantity_today[pd.isnull(fin_df.quantity_today)] = 0
        fin_df.quantity_week_ago[pd.isnull(fin_df.quantity_week_ago)] = 0

        stop_words = ['drink','bag','gift','card','sauce','dip','soda','coke','pepsi','sprite','cola','beer','beverage','extra','crumbles']


        #get rid of stop words
        fin_df.name = fin_df.name.str.lower()

        indexes = [True]*len(fin_df.name)

        for i, name in enumerate(fin_df.name):
            for word in stop_words:
                if word in name:
                    indexes[i]=False


        fin_df = fin_df[indexes]


        fin_df.name = fin_df.name.str.lower()

        indexes = [True]*len(fin_df.name)

        for i, name in enumerate(fin_df.name):
            for word in stop_words:
                if word in name:
                    indexes[i]=False


        fin_df = fin_df[indexes]
        #merge day and avg data to make the comparison possible
        fin_df_avg = fin_df_day.merge(fin_df_avg, on = ['name','id'], suffixes = ['_today','_avg'])
        #filter out all items that haven't sold at least 5 products
        fin_df_avg = fin_df_avg[fin_df_avg.quantity_avg >= 5]
        
        fin_df_avg.quantity_today[pd.isnull(fin_df_avg.quantity_today)] = ''
        fin_df_avg.quantity_avg[pd.isnull(fin_df_avg.quantity_avg)] = 0


        fin_df_avg.name = fin_df_avg.name.str.lower()

        indexes = [True]*len(fin_df_avg.name)

        for i, name in enumerate(fin_df_avg.name):
            for word in stop_words:
                if word in name:
                    indexes[i]=False
        
        fin_df_avg.quantity_avg = fin_df_avg.quantity_avg[:]/7
        fin_df_avg.price_avg = fin_df_avg.price_avg.divide(7)

        fin_df_avg = fin_df_avg[indexes]
        #difference the quantity and revenue of today and a week ago, this will give us a differential that has meaning
        fin_df['quant_difference'] = (fin_df.quantity_today-fin_df.quantity_week_ago)
        fin_df['rev_difference'] = (fin_df.price_today-fin_df.price_week_ago)

        fin_df_avg['quant_difference'] = (fin_df_avg.quantity_today-fin_df_avg.quantity_avg)
        fin_df_avg['rev_difference'] = (fin_df_avg.price_today-fin_df_avg.price_avg)

        if np.sum(fin_df.price_today) != 0 or np.sum(fin_df.price_week_ago) != 0:
            #let them know how they are doing with total sales figures (revenue based)
            if np.sum(fin_df.rev_difference) > -20 and np.sum(fin_df.rev_difference) < 100:
                print('Business as usual compared to last week\'s business on the same day.')
            elif np.sum(fin_df.rev_difference) > 100:
                print('You are doing very well this hour compared to last week\'s business on the same day.')
            elif np.sum(fin_df.rev_difference) < -20 and np.sum(fin_df.rev_difference) > -100:
                print('Business is slower than usual compared to last week\'s business on the same day.')
            else:
                print('Business is very slow this hour compared to last week\'s business on the same day.')


        if np.sum(fin_df.price_today) != 0 or np.sum(fin_df.price_avg) != 0:
            if np.sum(fin_df_avg.rev_difference) > -20 and np.sum(fin_df_avg.rev_difference) < 100:
                print('Business as usual compared to the weekly average.')
            elif np.sum(fin_df_avg.rev_difference) > 100:
                print('You are doing very well this hour compared to the week-long average')
            elif np.sum(fin_df_avg.rev_difference) < -20 and np.sum(fin_df_avg.rev_difference) > -100:
                print('Business is slower than usual compared to the week-long average')
            else:
                print('Business is very slow this hour compared to the week-long average')

        #Let them know which item has the biggest sales differential    
        #if np.sum(fin_df.quantity_today) != 0 or np.sum(fin_df.quantity_week_ago) != 0:
            #print('This item has the biggest negative differential from last week: %s'%fin_df.name.iloc[np.argmax(fin_df.quant_difference)])
        #if np.sum(fin_df.quantity_today) != 0 or np.sum(fin_df.quantity_avg) != 0:
        #    print('This item has the biggest negative differential from the average for this hour: %s'%fin_df_avg.name.iloc[np.argmax(fin_df_avg.quant_difference)])
        #except:
        #    pass
        #fin_df_avg[np.argpartition(fin_df_avg.quantity_avg, 3)]
        ##slow_today = fin_df.nsmallest(3,'quantity_today')
        #slow_today = fin_df[np.argmin(fin_df.quantity_today, axis = 1)]

        #slow_past = fin_df[np.argmin(fin_df.quantity_week_ago, axis = 1)]
        
        #slow_avg = fin_df_avg[np.argmin(fin_df_avg.quantity_avg)]

        rev_avg = fin_df_avg.price_avg
        rev_today = fin_df_avg.price_today
        rev_week_ago = fin_df.price_week_ago

        fin_df_avg_bottom_3 = fin_df_avg.iloc[-3:,:]

        #fin_df_avg_bottom_3 = fin_df_avg.merge(df2_name_url,on = 'name', how = 'outer')
        #print rev_today
        #print rev_week_ago
        bottom_3_dict = fin_df_avg_bottom_3.to_dict('records')
        #slow_today
        #slow_past
        #print(slow_avg.head())

        record.update_many({
              'merchant_uid': MERCHANT_ID
                },{
                '$set': {
                    'Need_to_Pump_products':bottom_3_dict
                   
                      }
                }, upsert=True)
    except:
        pass

/Users/jackdwyer/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:594: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jackdwyer/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/jackdwyer/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/jackdwyer

Business is very slow this hour compared to last week's business on the same day.
Business is very slow this hour compared to the week-long average
               name                                    id  quantity_today  \
11  tofu balls 3pcs  9a83c819-8098-4deb-8e01-6354b9a2c492             3.0   
12         combo #4  a07a435c-c863-4fdc-b400-93cdcbef9066             2.0   
14     lg 14pcs kfc  8ad6e9cc-21ce-49aa-8edb-0f0dfacf4bb2             1.0   

    price_today  quantity_avg   price_avg  quant_difference  rev_difference  
11         13.0     46.714286  202.428571        -43.714286     -189.428571  
12         23.5     31.142857  365.928571        -29.142857     -342.428571  
14         18.0     15.571429  280.285714        -14.571429     -262.285714  
Business is slower than usual compared to last week's business on the same day.
Business is very slow this hour compared to the week-long average
                             name                                    id  \
53        